In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        dataset_path = os.path.join(dirname, filename)
        df = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MTeamSpellings.csv', sep=";", encoding='cp1252')

In [2]:
# INFO: This notebook creates 4 files. There are two files for each gender
# The MData (and WData) file has the overall average or percentage of each stat (FG, FG3, FT, etc.)
# for each team for each year. The averages are taken from both the regular season stats file
# AND the tourney stats file. (in the OG kaggle notebook we only considered regular season stats)

# The other two data files (MGamesFull and WGamesFull) contain ALL the different
# games played in past years with the each teams overall stats for that season
# the last column shows who won that matchup
# 1 = left team won
# 0 = right team won
# the left team stats are in the left half of the stats in the data file. (as for right team)
# These two files don't have the teamID because we are trying to predict the winner
# of a game just depending on the statistics for that team

# I have tested the logistic regression model on that data, and it is like 65 % accurate (meh)
# We just have to create a function that takes in each round and then performs linear regression
# to predict who wins

In [3]:
city_data = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/Cities.csv")
Conferences = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/Conferences.csv")
Games = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MConferenceTourneyGames.csv")
MTourney = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyDetailedResults.csv")
WTourney = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv")
MSeeds = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeeds.csv")
WSeeds = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySeeds.csv")
MRegular = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv")
WRegular = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonDetailedResults.csv")
MTeams = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeams.csv")
WTeams = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WTeams.csv")
sample_submission = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv")
current_seeds = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv")

In [4]:
#Men's Data

MData = pd.DataFrame()
WIndex = [0, 2, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
LIndex = [0, 4, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]


for team in MTeams["TeamID"]:
    aux = pd.DataFrame()
    for year in range(2003, 2024):        
        #Tourney Win
        aux1 = pd.DataFrame()
        aux1 = MTourney[ (MTourney["Season"] == year) & (MTourney["WTeamID"] == team)]
        aux1 = aux1.iloc[:, WIndex]
        aux1.rename(columns={"WTeamID":"TeamID", "WFGM":"FGM", "WFGA":"FGA", "WFGM3":"FGM3", "WFGA3":"FGA3", "WFTM":"FTM", "WFTA":"FTA", "WOR":"OR",
                             "WDR":"DR", "WAst":"Ast", "WTO":"TO", "WStl":"Stl", "WBlk":"Blk","WPF":"PF"}, inplace=True)
        if not len(aux1.index) == 0:
            aux = pd.concat([aux, aux1.copy()])
        #Regular Season Win
        aux1 = pd.DataFrame()
        aux1 = MRegular[ (MRegular["Season"] == year) & (MRegular["WTeamID"] == team)]
        aux1 = aux1.iloc[:, WIndex]
        aux1.rename(columns={"WTeamID":"TeamID", "WFGM":"FGM", "WFGA":"FGA", "WFGM3":"FGM3", "WFGA3":"FGA3", "WFTM":"FTM", "WFTA":"FTA", "WOR":"OR",
                             "WDR":"DR", "WAst":"Ast", "WTO":"TO", "WStl":"Stl", "WBlk":"Blk","WPF":"PF"}, inplace=True)
        if not len(aux1.index) == 0:
            aux = pd.concat([aux, aux1.copy()])
        
        #Tourney Loss
        aux1 = pd.DataFrame()
        aux1 = MTourney[ (MTourney["Season"] == year) & (MTourney["LTeamID"] == team)]
        aux1 = aux1.iloc[:, LIndex]
        aux1.rename(columns={"LTeamID":"TeamID", "LFGM":"FGM", "LFGA":"FGA", "LFGM3":"FGM3", "LFGA3":"FGA3", "LFTM":"FTM", "LFTA":"FTA", "LOR":"OR",
                             "LDR":"DR", "LAst":"Ast", "LTO":"TO", "LStl":"Stl", "LBlk":"Blk","LPF":"PF"}, inplace=True)
        if not len(aux1.index) == 0:
            aux = pd.concat([aux, aux1.copy()])
        #Regular Season Loss
        aux1 = pd.DataFrame()
        aux1 = MRegular[ (MRegular["Season"] == year) & (MRegular["LTeamID"] == team)]
        aux1 = aux1.iloc[:, LIndex]
        aux1.rename(columns={"LTeamID":"TeamID", "LFGM":"FGM", "LFGA":"FGA", "LFGM3":"FGM3", "LFGA3":"FGA3", "LFTM":"FTM", "LFTA":"FTA", "LOR":"OR",
                             "LDR":"DR", "LAst":"Ast", "LTO":"TO", "LStl":"Stl", "LBlk":"Blk","LPF":"PF"}, inplace=True)
        if not len(aux1.index) == 0:
            aux = pd.concat([aux, aux1.copy()])
        
    if not len(aux.index) == 0:
        aux = aux.groupby(["Season"]).mean()
        MData = pd.concat([MData, aux.copy()])
    #print("--"*50)
    #print(f"Done with team {team}.")
    #print("--"*50)
    

print(MData)
    
        

        TeamID        FGM        FGA      FGM3       FGA3        FTM  \
Season                                                                 
2014    1101.0  20.333333  50.142857  6.666667  17.857143  15.809524   
2015    1101.0  21.428571  52.928571  7.250000  19.178571  10.892857   
2016    1101.0  23.814815  53.925926  6.851852  18.851852  15.370370   
2017    1101.0  24.440000  53.320000  7.080000  19.080000  11.920000   
2018    1101.0  25.518519  56.481481  6.481481  19.962963  13.148148   
...        ...        ...        ...       ...        ...        ...   
2023    1473.0  24.206897  57.172414  7.379310  20.379310  11.827586   
2023    1474.0  25.666667  58.933333  9.033333  25.966667  15.266667   
2023    1475.0  26.172414  61.310345  9.551724  26.103448  13.931034   
2023    1476.0  22.900000  52.433333  7.533333  21.633333  12.800000   
2023    1477.0  25.593750  56.687500  8.250000  25.843750  11.218750   

              FTA         OR         DR        Ast         TO  

In [5]:
#Women's Data

WData = pd.DataFrame()
WIndex = [0, 2, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
LIndex = [0, 4, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]


for team in WTeams["TeamID"]:
    aux = pd.DataFrame()
    for year in range(2003, 2024):        
        #Tourney Win
        aux1 = pd.DataFrame()
        aux1 = WTourney[ (WTourney["Season"] == year) & (WTourney["WTeamID"] == team)]
        aux1 = aux1.iloc[:, WIndex]
        aux1.rename(columns={"WTeamID":"TeamID", "WFGM":"FGM", "WFGA":"FGA", "WFGM3":"FGM3", "WFGA3":"FGA3", "WFTM":"FTM", "WFTA":"FTA", "WOR":"OR",
                             "WDR":"DR", "WAst":"Ast", "WTO":"TO", "WStl":"Stl", "WBlk":"Blk","WPF":"PF"}, inplace=True)
        if not len(aux1.index) == 0:
            aux = pd.concat([aux, aux1.copy()])
        #Regular Season Win
        aux1 = pd.DataFrame()
        aux1 = WRegular[ (WRegular["Season"] == year) & (WRegular["WTeamID"] == team)]
        aux1 = aux1.iloc[:, WIndex]
        aux1.rename(columns={"WTeamID":"TeamID", "WFGM":"FGM", "WFGA":"FGA", "WFGM3":"FGM3", "WFGA3":"FGA3", "WFTM":"FTM", "WFTA":"FTA", "WOR":"OR",
                             "WDR":"DR", "WAst":"Ast", "WTO":"TO", "WStl":"Stl", "WBlk":"Blk","WPF":"PF"}, inplace=True)
        if not len(aux1.index) == 0:
            aux = pd.concat([aux, aux1.copy()])
        
        #Tourney Loss
        aux1 = pd.DataFrame()
        aux1 = WTourney[ (WTourney["Season"] == year) & (WTourney["LTeamID"] == team)]
        aux1 = aux1.iloc[:, LIndex]
        aux1.rename(columns={"LTeamID":"TeamID", "LFGM":"FGM", "LFGA":"FGA", "LFGM3":"FGM3", "LFGA3":"FGA3", "LFTM":"FTM", "LFTA":"FTA", "LOR":"OR",
                             "LDR":"DR", "LAst":"Ast", "LTO":"TO", "LStl":"Stl", "LBlk":"Blk","LPF":"PF"}, inplace=True)
        if not len(aux1.index) == 0:
            aux = pd.concat([aux, aux1.copy()])
        #Regular Season Loss
        aux1 = pd.DataFrame()
        aux1 = WRegular[ (WRegular["Season"] == year) & (WRegular["LTeamID"] == team)]
        aux1 = aux1.iloc[:, LIndex]
        aux1.rename(columns={"LTeamID":"TeamID", "LFGM":"FGM", "LFGA":"FGA", "LFGM3":"FGM3", "LFGA3":"FGA3", "LFTM":"FTM", "LFTA":"FTA", "LOR":"OR",
                             "LDR":"DR", "LAst":"Ast", "LTO":"TO", "LStl":"Stl", "LBlk":"Blk","LPF":"PF"}, inplace=True)
        if not len(aux1.index) == 0:
            aux = pd.concat([aux, aux1.copy()])
        
    if not len(aux.index) == 0:
        aux = aux.groupby(["Season"]).mean()
        WData = pd.concat([WData, aux.copy()])
    #print("--"*50)
    #print(f"Done with team {team}.")
    #print("--"*50)
    

print(WData)
    
        

        TeamID        FGM        FGA      FGM3       FGA3        FTM  \
Season                                                                 
2014    3101.0  25.086957  59.565217  8.521739  26.652174  11.304348   
2015    3101.0  21.720000  57.600000  7.280000  26.040000  14.320000   
2016    3101.0  24.560000  58.320000  7.680000  23.080000  16.400000   
2017    3101.0  23.962963  57.777778  8.481481  25.555556  14.481481   
2018    3101.0  22.615385  54.038462  8.461538  25.423077  12.230769   
...        ...        ...        ...       ...        ...        ...   
2023    3473.0  19.760000  50.320000  6.400000  21.000000   8.720000   
2023    3474.0  20.038462  58.076923  4.423077  16.692308  12.615385   
2023    3475.0  21.807692  55.230769  4.423077  15.538462  14.346154   
2023    3476.0  21.571429  55.821429  6.178571  20.250000   9.857143   
2023    3477.0  22.937500  58.218750  3.656250  14.250000  15.093750   

              FTA         OR         DR        Ast         TO  

In [6]:
MData['TeamID'] = MData['TeamID'].astype(int)
WData['TeamID'] = WData['TeamID'].astype(int)

print(MData)
print(WData)

MData.to_csv("/kaggle/working/MData.csv", index=False)
WData.to_csv("/kaggle/working/WData.csv", index=False)

        TeamID        FGM        FGA      FGM3       FGA3        FTM  \
Season                                                                 
2014      1101  20.333333  50.142857  6.666667  17.857143  15.809524   
2015      1101  21.428571  52.928571  7.250000  19.178571  10.892857   
2016      1101  23.814815  53.925926  6.851852  18.851852  15.370370   
2017      1101  24.440000  53.320000  7.080000  19.080000  11.920000   
2018      1101  25.518519  56.481481  6.481481  19.962963  13.148148   
...        ...        ...        ...       ...        ...        ...   
2023      1473  24.206897  57.172414  7.379310  20.379310  11.827586   
2023      1474  25.666667  58.933333  9.033333  25.966667  15.266667   
2023      1475  26.172414  61.310345  9.551724  26.103448  13.931034   
2023      1476  22.900000  52.433333  7.533333  21.633333  12.800000   
2023      1477  25.593750  56.687500  8.250000  25.843750  11.218750   

              FTA         OR         DR        Ast         TO  

In [7]:
MData.head(20)

,TeamID,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
Season,,,,,,,,,,,,,,
2014,1101,20.333333,50.142857,6.666667,17.857143,15.809524,21.190476,8.000000,20.333333,10.000000,15.000000,5.761905,1.476190,21.571429
2015,1101,21.428571,52.928571,7.250000,19.178571,10.892857,14.964286,8.250000,19.642857,11.857143,12.821429,5.928571,1.178571,20.607143
2016,1101,23.814815,53.925926,6.851852,18.851852,15.370370,21.740741,8.185185,22.518519,12.888889,13.407407,6.740741,2.444444,22.370370
2017,1101,24.440000,53.320000,7.080000,19.080000,11.920000,18.560000,7.560000,22.880000,13.600000,14.480000,7.000000,2.760000,22.160000
2018,1101,25.518519,56.481481,6.481481,19.962963,13.148148,18.666667,9.037037,23.222222,13.888889,14.407407,7.148148,3.629630,21.037037
2019,1101,25.066667,55.166667,7.166667,19.033333,13.500000,18.766667,8.866667,22.600000,14.433333,11.600000,8.033333,2.566667,19.033333
2020,1101,24.592593,56.074074,6.481481,20.592593,17.111111,23.037037,10.037037,21.814815,13.592593,14.740741,9.000000,2.666667,22.000000
2021,1101,26.480000,59.280000,7.760000,21.320000,13.480000,19.400000,9.640000,22.600000,17.600000,13.080000,9.160000,2.840000,6.480000
2022,1101,24.827586,58.896552,6.689655,21.310345,16.827586,22.655172,8.758621,20.068966,14.068966,12.000000,10.448276,1.724138,22.379310


In [8]:
#Men's Games
MGames = pd.DataFrame()


###Tourney Games
MGamesList1 = MTourney[MTourney["Season"] > 2003]
MGamesList1 = MGamesList1[["Season", "WTeamID"]]
MGamesList1["Season"] = MGamesList1["Season"]-1
MGamesList2 = MTourney[MTourney["Season"] > 2003]
MGamesList2 = MGamesList2[["Season", "LTeamID"]]
MGamesList2["Season"] = MGamesList2["Season"]-1

WJUJU = pd.merge(MGamesList1, MData, how="left", left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"])#, how="outer")
LJUJU = pd.merge(MGamesList2, MData, how="left", left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"])#, how="outer")
WJUJU = WJUJU.iloc[:, 3:]
LJUJU = LJUJU.iloc[:, 3:]
#print(WJUJU)
#print(LJUJU)
#MGames = pd.concat([WJUJU, LJUJU], axis = 1)
AUX = pd.concat([WJUJU, LJUJU], axis = 1)
AUX["Winner"] = 1
MGames = pd.concat([MGames, AUX])
AUX = pd.concat([LJUJU, WJUJU], axis = 1)
AUX["Winner"] = 0
MGames = pd.concat([MGames, AUX])




###Season Games
MGamesList1 = MRegular[MRegular["Season"] > 2003]
MGamesList1 = MGamesList1[["Season", "WTeamID"]]
MGamesList1["Season"] = MGamesList1["Season"]-1
MGamesList2 = MRegular[MRegular["Season"] > 2003]
MGamesList2 = MGamesList2[["Season", "LTeamID"]]
MGamesList2["Season"] = MGamesList2["Season"]-1

WJUJU = pd.merge(MGamesList1, MData, how="left", left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"])#, how="outer")
LJUJU = pd.merge(MGamesList2, MData, how="left", left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"])#, how="outer")
WJUJU = WJUJU.iloc[:, 3:]
LJUJU = LJUJU.iloc[:, 3:]
#print(WJUJU)
#print(LJUJU)
#MGames = pd.concat([WJUJU, LJUJU], axis = 1)
AUX = pd.concat([WJUJU, LJUJU], axis = 1)
AUX["Winner"] = 1
MGames = pd.concat([MGames, AUX])
AUX = pd.concat([LJUJU, WJUJU], axis = 1)
AUX["Winner"] = 0
MGames = pd.concat([MGames, AUX])

###Done

print(MGames)
print(MGames.dropna())

MGames = MGames.dropna()

MGames.to_csv("/kaggle/working/MGamesFull.csv", index=False)



              FGM        FGA      FGM3       FGA3        FTM        FTA  \
0       23.482759  54.517241  8.275862  21.310345  16.241379  24.000000   
1       23.965517  57.000000  6.310345  19.586207  14.793103  20.758621   
2       29.212121  62.333333  5.939394  15.393939  15.090909  22.303030   
3       23.357143  53.107143  4.428571  13.321429  15.892857  21.857143   
4       27.181818  60.030303  7.393939  20.363636  19.363636  27.181818   
...           ...        ...       ...        ...        ...        ...   
108620  25.343750  57.656250  6.750000  21.156250  14.500000  20.218750   
108621  24.685714  53.342857  6.057143  17.428571  15.428571  22.171429   
108622  23.774194  57.354839  8.129032  23.225806   9.258065  13.806452   
108623  23.562500  55.437500  7.968750  24.156250  14.500000  19.156250   
108624  25.259259  57.666667  8.000000  23.037037  11.037037  17.185185   

               OR         DR        Ast         TO  ...        FTM        FTA  \
0       13.448276 

In [9]:
#Women's Games
WGames = pd.DataFrame()


###Tourney Games
MGamesList1 = WTourney[WTourney["Season"] > 2003]
MGamesList1 = MGamesList1[["Season", "WTeamID"]]
MGamesList1["Season"] = MGamesList1["Season"]-1
MGamesList2 = WTourney[WTourney["Season"] > 2003]
MGamesList2 = MGamesList2[["Season", "LTeamID"]]
MGamesList2["Season"] = MGamesList2["Season"]-1

WJUJU = pd.merge(MGamesList1, WData, how="left", left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"])#, how="outer")
LJUJU = pd.merge(MGamesList2, WData, how="left", left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"])#, how="outer")
WJUJU = WJUJU.iloc[:, 3:]
LJUJU = LJUJU.iloc[:, 3:]
#print(WJUJU)
#print(LJUJU)
#MGames = pd.concat([WJUJU, LJUJU], axis = 1)
AUX = pd.concat([WJUJU, LJUJU], axis = 1)
AUX["Winner"] = 1
WGames = pd.concat([WGames, AUX])
AUX = pd.concat([LJUJU, WJUJU], axis = 1)
AUX["Winner"] = 0
WGames = pd.concat([WGames, AUX])




###Season Games
MGamesList1 = WRegular[WRegular["Season"] > 2003]
MGamesList1 = MGamesList1[["Season", "WTeamID"]]
MGamesList1["Season"] = MGamesList1["Season"]-1
MGamesList2 = WRegular[WRegular["Season"] > 2003]
MGamesList2 = MGamesList2[["Season", "LTeamID"]]
MGamesList2["Season"] = MGamesList2["Season"]-1

WJUJU = pd.merge(MGamesList1, WData, how="left", left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"])#, how="outer")
LJUJU = pd.merge(MGamesList2, WData, how="left", left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"])#, how="outer")
WJUJU = WJUJU.iloc[:, 3:]
LJUJU = LJUJU.iloc[:, 3:]
#print(WJUJU)
#print(LJUJU)
#MGames = pd.concat([WJUJU, LJUJU], axis = 1)
AUX = pd.concat([WJUJU, LJUJU], axis = 1)
AUX["Winner"] = 1
WGames = pd.concat([WGames, AUX])
AUX = pd.concat([LJUJU, WJUJU], axis = 1)
AUX["Winner"] = 0
WGames = pd.concat([WGames, AUX])

###Done

##print(WGames)
print(WGames.dropna())
WGames = WGames.dropna()

WGames.to_csv("/kaggle/working/WGamesFull.csv", index=False)


             FGM        FGA      FGM3       FGA3        FTM        FTA  \
63     25.727273  61.151515  4.939394  17.000000  11.909091  15.636364   
64     26.305556  63.138889  4.333333  14.222222  13.527778  20.138889   
65     23.696970  58.484848  2.696970  10.454545  15.727273  22.121212   
66     31.500000  66.470588  4.882353  14.970588  12.088235  16.411765   
67     25.611111  61.555556  5.194444  16.333333  17.527778  24.750000   
...          ...        ...       ...        ...        ...        ...   
76191  22.750000  56.250000  7.892857  24.857143   8.964286  13.107143   
76192  26.633333  57.133333  6.400000  19.266667  15.433333  22.133333   
76193  25.709677  63.096774  4.161290  14.258065  12.645161  19.516129   
76194  24.838710  61.483871  6.580645  21.129032  11.612903  15.870968   
76195  25.718750  56.000000  6.656250  16.906250  10.843750  15.062500   

              OR         DR        Ast         TO  ...        FTM        FTA  \
63     14.090909  25.303030  15

In [10]:
# SETTING UP THE FORMATTING OF THE BRACKET, SLOTS, ROUNDS, MATCHES, AND SEEDS
initial_slots = [
    'R1W1', 'R1W2', 'R1W3', 'R1W4', 'R1W5', 'R1W6', 'R1W7', 'R1W8',
    'R1X1', 'R1X2', 'R1X3', 'R1X4', 'R1X5', 'R1X6', 'R1X7', 'R1X8',
    'R1Y1', 'R1Y2', 'R1Y3', 'R1Y4', 'R1Y5', 'R1Y6', 'R1Y7', 'R1Y8',
    'R1Z1', 'R1Z2', 'R1Z3', 'R1Z4', 'R1Z5', 'R1Z6', 'R1Z7', 'R1Z8'
]


# Define the 
initial_matchups = {
    'R1W1': (1, 16),
    'R1W2': (2, 15),
    'R1W3': (3, 14),
    'R1W4': (4, 13),
    'R1W5': (5, 12),
    'R1W6': (6, 11),
    'R1W7': (7, 10),
    'R1W8': (8, 9),
    'R1X1': (1, 16),
    'R1X2': (2, 15),
    'R1X3': (3, 14),
    'R1X4': (4, 13),
    'R1X5': (5, 12),
    'R1X6': (6, 11),
    'R1X7': (7, 10),
    'R1X8': (8, 9),
    'R1Y1': (1, 16),
    'R1Y2': (2, 15),
    'R1Y3': (3, 14),
    'R1Y4': (4, 13),
    'R1Y5': (5, 12),
    'R1Y6': (6, 11),
    'R1Y7': (7, 10),
    'R1Y8': (8, 9),
    'R1Z1': (1, 16),
    'R1Z2': (2, 15),
    'R1Z3': (3, 14),
    'R1Z4': (4, 13),
    'R1Z5': (5, 12),
    'R1Z6': (6, 11),
    'R1Z7': (7, 10),
    'R1Z8': (8, 9),
}


round2_map = {
        'R1W1': 'R2W1', 'R1W8': 'R2W1',
        'R1W2': 'R2W2', 'R1W7': 'R2W2',
        'R1W3': 'R2W3', 'R1W6': 'R2W3',
        'R1W4': 'R2W4', 'R1W5': 'R2W4',
        'R1X1': 'R2X1', 'R1X8': 'R2X1',
        'R1X2': 'R2X2', 'R1X7': 'R2X2',
        'R1X3': 'R2X3', 'R1X6': 'R2X3',
        'R1X4': 'R2X4', 'R1X5': 'R2X4',
        'R1Y1': 'R2Y1', 'R1Y8': 'R2Y1',
        'R1Y2': 'R2Y2', 'R1Y7': 'R2Y2',
        'R1Y3': 'R2Y3', 'R1Y6': 'R2Y3',
        'R1Y4': 'R2Y4', 'R1Y5': 'R2Y4',
        'R1Z1': 'R2Z1', 'R1Z8': 'R2Z1',
        'R1Z2': 'R2Z2', 'R1Z7': 'R2Z2',
        'R1Z3': 'R2Z3', 'R1Z6': 'R2Z3',
        'R1Z4': 'R2Z4', 'R1Z5': 'R2Z4',
    }

round3_map = {
    'R2W1': 'R3W1', 'R2W2': 'R3W1',
    'R2W3': 'R3W2', 'R2W4': 'R3W2',
    'R2X1': 'R3X1', 'R2X2': 'R3X1',
    'R2X3': 'R3X2', 'R2X4': 'R3X2',
    'R2Y1': 'R3Y1', 'R2Y2': 'R3Y1',
    'R2Y3': 'R3Y2', 'R2Y4': 'R3Y2',
    'R2Z1': 'R3Z1', 'R2Z2': 'R3Z1',
    'R2Z3': 'R3Z2', 'R2Z4': 'R3Z2',
}

round4_map = {
    'R3W1': 'R4W1', 'R3W2': 'R4W1',
    'R3X1': 'R4X1', 'R3X2': 'R4X1',
    'R3Y1': 'R4Y1', 'R3Y2': 'R4Y1',
    'R3Z1': 'R4Z1', 'R3Z2': 'R4Z1',
}

round5_map = {
    'R4W1': 'R5WX', 'R4X1': 'R5WX',
    'R4Y1': 'R5YZ', 'R4Z1': 'R5YZ',
}

round6_map = {
    'R5WX': 'R6CH',
    'R5YZ': 'R6CH',
}


In [11]:
# Logistic Regression for Men
MGamesFull = pd.read_csv("/kaggle/working/MGamesFull.csv")

X = MGamesFull.drop('Winner', axis=1) # Predictors
# .drop is used to drop specified labels, axis=1 means a column, and axis=0 means a row
y = MGamesFull['Winner'] # Response variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(173848, 26)
(43462, 26)
(173848,)
(43462,)


In [12]:
# Fitting a logistic regression model for the Men's data
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# These predictions are binary
print(y_pred)

[1 0 1 ... 1 1 1]


In [13]:
# Make predictions on the testing set
predictions_proba = model.predict_proba(X_test)[:, 1]

# Evaluate the model using the Brier score
brier_score_val = brier_score_loss(y_test, predictions_proba)
print(f"Brier Score for Men's: {brier_score_val}")

Brier Score for Men's: 0.2184301774758122


In [14]:
# Generating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
# Calculating the overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Overall Accuracy: {accuracy}")
# Calculating sensitivity (recall)
sensitivity = recall_score(y_test, y_pred)
print(f"Sensitivity: {sensitivity}")
# Calculating specificity
# Specificity = TN / (TN + FP)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
print(f"Specificity: {specificity}")

Confusion Matrix:
[[13918  7970]
 [ 7476 14098]]
Overall Accuracy: 0.6446090837973402
Sensitivity: 0.6534717715768981
Specificity: 0.6358735380116959


In [15]:
# Logistic Regression for the Women's data
WGamesFull = pd.read_csv("/kaggle/working/WGamesFull.csv")

X = WGamesFull.drop('Winner', axis=1) # Predictors
# .drop is used to drop specified labels, axis=1 means a column, and axis=0 means a row
y = WGamesFull['Winner'] # Response variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print(X_train.shape)
print(X_test.shape)

(113835, 26)
(28459, 26)


In [16]:
# Fitting a logistic regression model for the Women's data
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# These predictions are binary
print(y_pred)

# In the MGamesFull we have the left team and right team stat for 
# left team won = 1
# right team won = 0

[0 1 0 ... 1 1 0]


In [17]:
# Make predictions on the testing set
predictions_proba = model.predict_proba(X_test)[:, 1]

# Evaluate the model using the Brier score
brier_score_val = brier_score_loss(y_test, predictions_proba)
print(f"Brier Score for Women's: {brier_score_val}")

Brier Score for Women's: 0.20678536226521876


In [18]:
# Generating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
# Calculating the overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Overall Accuracy: {accuracy}")
# Calculating sensitivity (recall)
sensitivity = recall_score(y_test, y_pred)
print(f"Sensitivity: {sensitivity}")
# Calculating specificity
# Specificity = TN / (TN + FP)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
print(f"Specificity: {specificity}")

Confusion Matrix:
[[9600 4597]
 [4732 9530]]
Overall Accuracy: 0.6721950876699814
Sensitivity: 0.6682092273173468
Specificity: 0.6761991970134535


In [19]:
# TODO: 
#still working on this cell..... 
# using previous years performance stat we're going to predict the outcomes for this year

MGames = pd.read_csv("/kaggle/working/MGamesFull.csv")
WGames = pd.read_csv("/kaggle/working/WGamesFull.csv")

MData = pd.read_csv("/kaggle/working/MData.csv")
WData = pd.read_csv("/kaggle/working/WData.csv")
current_seeds = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv")

current_seeds['Region_Seed'] = current_seeds['Seed']
current_seeds['Region'] = current_seeds['Seed'].apply(lambda x: (x[0]))
current_seeds['Seed'] = current_seeds['Seed'].apply(lambda x: int(x[1:3]))
mens_seeds = current_seeds[current_seeds['Tournament'] == 'M']
womens_seeds = current_seeds[current_seeds['Tournament'] == 'W']

def get_matchups_for_next_round(previous_round_winners, round_map):
    """
    Given the winners from the previous round and a mapping of previous round slots to the next round slots,
    return a list of matchups for the next round.
    """
    next_round_matchups = {}
    for prev_slot, winner in previous_round_winners.items():
        if prev_slot in round_map:
            next_slot = round_map[prev_slot]
            if next_slot in next_round_matchups:
                next_round_matchups[next_slot].append(winner)
            else:
                next_round_matchups[next_slot] = [winner]
    return [(slot, teams[0], teams[1]) for slot, teams in next_round_matchups.items() if len(teams) == 2]


def predict_match_winner(left_team, right_team, gender, model):
    """
    Predict the match winner based on seed difference using the logistic regression model.
    """
    if gender == "W":
        left_team_stats = MGames.iloc[:, 0:12]
        right_team_stats = WGames.iloc[:, 13:]
        predictions_proba = model.predict_proba(X_test)[:, 1]
    else:
        left_team_stats = WGames.iloc[:, 0:12]
        right_team_stats = WGames.iloc[:, 13:]
        predictions_proba = model.predict_proba(X_test)[:, 1]

    # Assuming higher probability indicates team1 wins
    return 1 if predictions_proba > 0.5 else 2


# given the seeds
# for each game given look at the two team IDs,
# we then need to retrieve the 2023 stats from Mdata file
# feed that into the model. that will tell us all the teams that won

def simulate_tournament(seeds, model, initial_slots):
    seeds['Region_Seed'] = seeds['Region'] + seeds['Seed'].astype(str).str.zfill(2)
    seeds = seeds.set_index('Region_Seed', drop=False)
    # Initialize DataFrame to record match outcomes
    match_outcomes = pd.DataFrame(columns=['Slot', 'Winner'])
    current_round_winners = {}
    
    for slot, (seed1, seed2) in initial_matchups.items():
        region = slot[2]  # Extract region from slot, e.g., 'W' from 'R1W1'
        team1_seed, team2_seed = f"{region}{seed1:02d}", f"{region}{seed2:02d}"
        
        if team1_seed in seeds['Region_Seed'].values and team2_seed in seeds['Region_Seed'].values:
            team1_id = seeds.loc[team1_seed, 'TeamID']
            team2_id = seeds.loc[team2_seed, 'TeamID']
            #for i in range(0,64):
                #if seeds['Tournament'][i] == 'M':
                 #   winner = predict_match_winner(seeds.loc[team1_seed, 'Seed'], seeds.loc[team2_seed, 'Seed'], "M", model)
               # else:
                    #winner = predict_match_winner(seeds.loc[team1_seed, 'Seed'], seeds.loc[team2_seed, 'Seed'], "W", model)
            winning_team = team1_id if winner == 1 else team2_id
            current_round_winners[slot] = winning_team
            match_outcomes = pd.concat([match_outcomes, pd.DataFrame([{'Slot': slot, 'Winner': winning_team}])], ignore_index=True)
            
    all_round_maps = [round2_map, round3_map, round4_map, round5_map, round6_map]
    seeds = seeds.set_index('TeamID', drop=False)

    for round_map in all_round_maps:
        next_round_matchups = get_matchups_for_next_round(current_round_winners, round_map)
        for slot, team1, team2 in next_round_matchups:
            winner = predict_match_winner(seeds.loc[team1, 'Seed'], seeds.loc[team2, 'Seed'], model)
            winning_team = team1 if winner == 1 else team2
            current_round_winners[slot] = winning_team
            match_outcomes = pd.concat([match_outcomes, pd.DataFrame([{'Slot': slot, 'Winner': winning_team}])], ignore_index=True)
            
    return match_outcomes

#mens_results = simulate_tournament(mens_seeds, model, initial_slots)
#womens_results = simulate_tournament(womens_seeds, model, initial_slots)

In [20]:
current_seeds = current_seeds.set_index('TeamID', drop=False)

# Function to get the winner team ID for a given slot and tournament type
#def get_winner_for_slot(tournament, slot, mens_results, womens_results):
    #if tournament == 'M':
        #winner_row = mens_results.loc[mens_results['Slot'] == slot]
    #else:  # Assuming 'W' for women's tournament
        #winner_row = womens_results.loc[womens_results['Slot'] == slot]
    # If there is a winner, return it, else return None (or a default value)
    #return winner_row['Winner'].values[0] if not winner_row.empty else None

# Update the Team column in the sample submission DataFrame
for index, row in sample_submission.iterrows():
    tournament = row['Tournament']
    slot = row['Slot']
    #winner = get_winner_for_slot(tournament, slot, mens_results, womens_results)
    #winner_slot = current_seeds.loc[winner,"Region_Seed"]
    #sample_submission.at[index, 'Team'] = winner_slot if winner_slot else sample_submission.at[index, 'Team']

In [21]:
sample_submission.to_csv('/kaggle/working/submission.csv', index=False)

In [22]:
sample_submission

,RowId,Tournament,Bracket,Slot,Team
0,0,M,1,R1W1,W01
1,1,M,1,R1W8,W08
2,2,M,1,R1W5,W05
3,3,M,1,R1W4,W04
4,4,M,1,R1W6,W06
...,...,...,...,...,...
121,121,W,1,R4Y1,Y01
122,122,W,1,R4Z1,Z01
123,123,W,1,R5WX,W01
124,124,W,1,R5YZ,Y01
